In [2]:
import numpy as np
import torch
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn

## Load Dataset

In [3]:
download_root = './MNIST_DATASET'

In [4]:
# Normalize data with mean=0.5, std=1.0
mnist_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])

In [5]:
train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

## Data Loader

In [6]:
train_loader = DataLoader(dataset=train_dataset, 
                         batch_size=20,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=20,
                         shuffle=True)

In [27]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(5*5*64, 1024)
        self.fc2 = nn.Linear(1024, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 5*5*64)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [28]:
model = CNN()

In [29]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    print("we are using GPU!")
    model = model.cuda()

we are using GPU!


## Optimizer 정의

In [31]:
optimizer = optim.Adam(model.parameters(), lr=0.01)

## Criterion 정의

In [32]:
criterion = nn.CrossEntropyLoss()

In [44]:
model.train()

for epoch in range(20):
    avg_loss = 0
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        output = model(x)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

Train Epoch: 0 [0/60000 (0%)]	Loss: 0.173111
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.150821
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.305165
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.073801
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.126338
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.101409
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.622615
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.188549
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.043731
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.013128
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.149328
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.043292
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.514427
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.551940
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.005374
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.216681
Train Epoch: 5 [20000/60000 (33%)]	Loss: 0.113799
Train Epoch: 5 [40000/60000 (67%)]	Loss: 0.419418
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.027641
Train Epoch: 6 [20000/60000 (33%)]	Loss: 1.215956
Train Epoch: 6 [40000/60000 (67%)]	

In [43]:
model.eval()

test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        if use_cuda:
            x, target = x.cuda(), target.cuda()
        output = model(x)
        test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 18.1590, Accuracy: 1007/10000 (10%)

